In [2]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE

  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [166]:
Pkg.test("FUSE")

     Testing FUSE
      Status `/private/var/folders/c3/035jbfxd5svg28gdfcc0y1c80000gn/T/jl_B1a7kM/Project.toml`
  [e64856f0] FUSE v0.1.0 `~/.julia/dev/FUSE`
  [a98d9a8b] Interpolations v0.13.4
  [682c06a0] JSON v0.21.2
  [c03570c3] Memoize v0.4.4
  [92933f4c] ProgressMeter v1.7.1
  [67601950] Quadrature v1.9.0
  [8dfed614] Test `@stdlib/Test`
      Status `/private/var/folders/c3/035jbfxd5svg28gdfcc0y1c80000gn/T/jl_B1a7kM/Manifest.toml`
  [621f4979] AbstractFFTs v1.0.1
  [79e6a3ab] Adapt v3.3.1
  [4fba245c] ArrayInterface v3.1.23
  [13072b0f] AxisAlgorithms v1.0.0
  [082447d4] ChainRules v0.8.24
  [d360d2e6] ChainRulesCore v0.10.13
  [861a8166] Combinatorics v1.0.2
  [38540f10] CommonSolve v0.2.0
  [bbf7d656] CommonSubexpressions v0.3.0
  [34da2185] Compat v3.32.0
  [187b0558] ConstructionBase v1.3.0
  [9a962f9c] DataAPI v1.7.0
  [864edb3b] DataStructures v0.18.10
  [e2d170a0] DataValueInterfaces v1.0.0
  [2b5f629d] DiffEqBase v6.70.0
  [163ba53b] DiffResults v1.0.3
  [b552c78f] DiffR

Test Summary: | Pass  Total
FDS           |   13     13
core_profiles.profiles_1d[:].grid
Test Summary: | Pass  Total
FDS_IMAS      |    5      5
Test Summary: | Pass  Total
JSON_IO       |    1      1
Test Summary:      | Pass  Total
parameters_set_get |   12     12
Test Summary:     | Pass  Total
parameters_access |    2      2
Test Summary:  | Pass  Total
functionarrays |    1      1


     Testing FUSE tests passed 


In [171]:
data = FUSE.dd();
resize!(data.core_profiles.profiles_1d,1)
#data.core_profiles.profiles_1d[1].grid.rho_tor_norm = Vector{Float64}(collect(1:10))
data.core_profiles.profiles_1d[1].electrons.temperature = Vector{Float64}(collect(1:10))

LoadError: Assign data to `core_profiles.profiles_1d[1].grid.rho_tor_norm` before assigning `core_profiles.profiles_1d[:].electrons.temperature`

In [170]:
filename = joinpath(dirname(dirname(abspath(@__FILE__))), "sample", "sample_equilibrium_ods.json")
@time data = FUSE.json2fuse(filename; verbose=false)

  0.001787 seconds (4.33 k allocations: 320.188 KiB)


EQUILIBRIUM
 time_slice
  [1]
   time ➡ Float64
   profiles_1d
    b_field_max ➡ 17-element Vector{Float64}
    dvolume_drho_tor ➡ 17-element Vector{Float64}
    gm9 ➡ 17-element Vector{Float64}
    dpsi_drho_tor ➡ 17-element Vector{Float64}
    surface ➡ 17-element Vector{Float64}
    rho_tor ➡ 17-element Vector{Float64}
    b_field_min ➡ 17-element Vector{Float64}
    darea_dpsi ➡ 17-element Vector{Float64}
    squareness_upper_inner ➡ 17-element Vector{Float64}
    squareness_lower_inner ➡ 17-element Vector{Float64}
    rho_tor_norm ➡ 17-element Vector{Float64}
    elongation ➡ 17-element Vector{Float64}
    b_field_average ➡ 17-element Vector{Float64}
    psi ➡ 17-element Vector{Float64}
    gm8 ➡ 17-element Vector{Float64}
    dpressure_dpsi ➡ 17-element Vector{Float64}
    triangularity_upper ➡ 17-element Vector{Float64}
    darea_drho_tor ➡ 17-element Vector{Float64}
    area ➡ 17-element Vector{Float64}
    trapped_fraction ➡ 17-element Vector{Float64}
    volume ➡ 17-element V